In [ ]:
# Package Installation and Kaggle Dataset Download
# Install required packages
!pip install kaggle tensorflow numpy opencv-python scikit-learn matplotlib

# Set Kaggle API credentials
import os
os.environ['KAGGLE_USERNAME'] = "your_kaggle_username"  # Replace with your username
os.environ['KAGGLE_KEY'] = "your_kaggle_api_key"        # Replace with your API key

# Download the dataset
!kaggle datasets download -d paultimothymooney/breast-histopathology-images

# Unzip the dataset
import zipfile
with zipfile.ZipFile("breast-histopathology-images.zip", "r") as zip_ref:
    zip_ref.extractall("breast-histopathology-images")


Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/breast-histopathology-images
License(s): CC0-1.0
 99% 3.07G/3.10G [00:31<00:00, 61.1MB/s]
100% 3.10G/3.10G [00:31<00:00, 107MB/s] 


In [ ]:
# Image Extraction from Dataset
import os
import cv2

def extract_images(dataset_path, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for root, _, files in os.walk(dataset_path):
        for file in files:
            if file.lower().endswith((".png", ".jpg", ".jpeg", ".bmp")):
                image_path = os.path.join(root, file)
                try:
                    img = cv2.imread(image_path)
                    if img is not None:
                        relative_path = os.path.relpath(root, dataset_path)
                        output_subdir = os.path.join(output_dir, relative_path)
                        os.makedirs(output_subdir, exist_ok=True)
                        output_path = os.path.join(output_subdir, file)
                        cv2.imwrite(output_path, img)
                        print(f"Extracted: {image_path} to {output_path}")
                    else:
                        print(f"Error reading image: {image_path}")
                except Exception as e:
                    print(f"Error processing image {image_path}: {e}")

dataset_path = "breast-histopathology-images/IDC_regular_ps50_idx5"
output_dir = "extracted_images"
extract_images(dataset_path, output_dir)


Streaming output truncated to the last 5000 lines.
Extracted: breast-histopathology-images/IDC_regular_ps50_idx5/10272/0/10272_idx5_x2751_y1801_class0.png to extracted_images/10272/0/10272_idx5_x2751_y1801_class0.png
Extracted: breast-histopathology-images/IDC_regular_ps50_idx5/10272/0/10272_idx5_x901_y551_class0.png to extracted_images/10272/0/10272_idx5_x901_y551_class0.png
Extracted: breast-histopathology-images/IDC_regular_ps50_idx5/10272/0/10272_idx5_x2201_y2051_class0.png to extracted_images/10272/0/10272_idx5_x2201_y2051_class0.png
Extracted: breast-histopathology-images/IDC_regular_ps50_idx5/10272/0/10272_idx5_x751_y801_class0.png to extracted_images/10272/0/10272_idx5_x751_y801_class0.png
Extracted: breast-histopathology-images/IDC_regular_ps50_idx5/10272/0/10272_idx5_x1001_y701_class0.png to extracted_images/10272/0/10272_idx5_x1001_y701_class0.png
Extracted: breast-histopathology-images/IDC_regular_ps50_idx5/10272/0/10272_idx5_x3201_y151_class0.png to extracted_images/10272/

In [ ]:
# Image Preprocessing and Path Loading
import cv2
import numpy as np
import os
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

IMG_SIZE = 128
BATCH_SIZE = 32

def preprocess_image(image_path):
    """Load, resize, and normalize an image."""
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is not None:
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = img / 255.0
        img = np.expand_dims(img, axis=-1)
        return img
    return None

def load_image_paths(dataset_path):
    """Load image paths and labels dynamically."""
    image_paths = []
    labels = []
    class_names = sorted(os.listdir(dataset_path))
    class_map = {name: i for i, name in enumerate(class_names)}
    for class_name in class_names:
        class_dir = os.path.join(dataset_path, class_name)
        if os.path.isdir(class_dir):
            for root, _, files in os.walk(class_dir):
                for file in files:
                    file_path = os.path.join(root, file)
                    if file.lower().endswith((".png", ".jpg", ".jpeg")):
                        image_paths.append(file_path)
                        labels.append(class_map[class_name])
    return image_paths, np.array(labels), class_names

dataset_path = "extracted_images"
image_paths, labels, class_names = load_image_paths(dataset_path)
X_train_paths, X_test_paths, y_train, y_test = train_test_split(image_paths, labels, test_size=0.2, random_state=42)


In [ ]:
#  Data Generator and CNN Model Definition
def image_data_generator(image_paths, labels, batch_size):
    while True:
        for i in range(0, len(image_paths), batch_size):
            batch_paths = image_paths[i:i+batch_size]
            batch_labels = labels[i:i+batch_size]
            images = [preprocess_image(path) for path in batch_paths if preprocess_image(path) is not None]
            labels_onehot = to_categorical(batch_labels, num_classes=len(class_names))
            if len(images) > 0:
                yield np.array(images), np.array(labels_onehot)

train_generator = image_data_generator(X_train_paths, y_train, BATCH_SIZE)
test_generator = image_data_generator(X_test_paths, y_test, BATCH_SIZE)

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def create_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2,2)),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(pool_size=(2,2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

input_shape = (IMG_SIZE, IMG_SIZE, 1)
num_classes = len(class_names)
model = create_cnn_model(input_shape, num_classes)
model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 57600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     7,372,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 279)            │        35,991 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,427,735 (28.33 MB)

 Trainable params: 7,427,735 (28.33 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Lightweight CNN Training with ImageDataGenerator
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

if tf.config.list_physical_devices('GPU'):
    print("GPU detected! Training will be faster.")
else:
    print("No GPU found. Training will use CPU.")

DATASET_PATH = "extracted_images"
IMG_SIZE = 64
BATCH_SIZE = 32
EPOCHS = 5

train_datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    color_mode='grayscale',
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    color_mode='grayscale',
    class_mode='categorical',
    subset='validation'
)

num_classes = len(train_generator.class_indices)

model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    MaxPooling2D(2, 2),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint = ModelCheckpoint("best_model.h5", save_best_only=True, monitor='val_loss', mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=EPOCHS,
    validation_data=val_generator,
    validation_steps=20,
    callbacks=[checkpoint, early_stopping]
)

model.save("final_model.h5")
print("Model training completed and saved successfully!")


No GPU found. Training will use CPU.
Found 222127 images belonging to 279 classes.
Found 55397 images belonging to 279 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 0.0154 - loss: 5.6214

100/100 ━━━━━━━━━━━━━━━━━━━━ 15s 132ms/step - accuracy: 0.0154 - loss: 5.6213 - val_accuracy: 0.0141 - val_loss: 5.5458
Epoch 2/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.0062 - loss: 5.5657

100/100 ━━━━━━━━━━━━━━━━━━━━ 13s 133ms/step - accuracy: 0.0062 - loss: 5.5656 - val_accuracy: 0.0094 - val_loss: 5.5334
Epoch 3/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - accuracy: 0.0106 - loss: 5.5166

100/100 ━━━━━━━━━━━━━━━━━━━━ 13s 134ms/step - accuracy: 0.0106 - loss: 5.5165 - val_accuracy: 0.0125 - val_loss: 5.4946
Epoch 4/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 20s 205ms/step - accuracy: 0.0094 - loss: 5.4948 - val_accuracy: 0.0125 - val_loss: 5.5339
Epoch 5/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step - accuracy: 0.0101 - loss: 5.4907

100/100 ━━━━━━━━━━━━━━━━━━━━ 13s 130ms/step - accuracy: 0.0101 - loss: 5.4907 - val_accuracy: 0.0078 - val_loss: 5.4786


Model training completed and saved successfully!
